In [1]:
!hostname
!pwd

t1v-n-d84a2c81-w-0
/home/patflick/dev/git/maxtext/MaxText


In [24]:
my_bucket = "gs://patflick-maxtext-lingvo"
base_output_directory= my_bucket + "/maxtext/2b/20231121"

checkpoint_path = "gs://test-example-123/maxtest-lg/2b-c4/v4-32/maxtext_cnndm/checkpoints"

extra_config = {
    # Run config
    "run_name": "prompt-2b-v1",
    "base_output_directory": base_output_directory,

    # Load checkpoint
    "load_from_other_directory": checkpoint_path,

    # Parallelism
    "dcn_tensor_parallelism": 4,
    "ici_tensor_parallelism": 4,

    # c4 dataset downloaded via `download_dataset.sh` from gs://allennlp-tensorflow-datasets/c4/en/3.0.1/
    "dataset_path": my_bucket + "/",
    "assets_path": "/home/patflick/dev/git/maxtext/assets", # TODO: move assets to GCS

    # Config for `decode_loop`
    "prompt": "Can you explain neural network inference?",
    "steps": 10,
    "max_predict_length": 50,

    "sampling_temperature": 2.0
}

import pyconfig
pyconfig.initialize(["", "configs/base-2b.yml"] + [f"{k}={v}" for k,v in extra_config.items()])
config = pyconfig.config
pyconfig._config.keys

OrderedDict([('run_name', 'prompt-2b-v1'),
             ('load_parameters_path', ''),
             ('load_from_other_directory',
              'gs://test-example-123/maxtest-lg/2b-c4/v4-32/maxtext_cnndm/checkpoints'),
             ('load_from_other_directory_step', -1),
             ('reuse_example_batch', 0),
             ('metrics_file', ''),
             ('gcs_metrics', False),
             ('dtype', dtype(bfloat16)),
             ('int8_training', False),
             ('use_dqdg', False),
             ('fwd_int8', True),
             ('dlhs_int8', True),
             ('drhs_int8', False),
             ('fwd_int8_qk', False),
             ('dlhs_int8_qk', False),
             ('drhs_int8_qk', False),
             ('fwd_int8_pv', True),
             ('dlhs_int8_pv', True),
             ('drhs_int8_pv', False),
             ('aqt_use_dummy_static_bound', False),
             ('aqt_use_fwd_quant', False),
             ('aqt_rng_type', 'jax.uniform'),
             ('global_parameter_sca

In [8]:
# MANUALLY load the training data and tokenizer
import max_utils  
import os
from jax.sharding import Mesh
from input_pipeline import create_data_iterator_with_tokenizer

# Mesh definition
devices_array = max_utils.create_device_mesh(config)
mesh = Mesh(devices_array, config.mesh_axes)

os.environ["TFDS_DATA_DIR"] = pyconfig.config.dataset_path + "/"
train_ds, sp_tokenizer = create_data_iterator_with_tokenizer(config, mesh)


Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]
Model path: /home/patflick/dev/git/maxtext/assets/tokenizer


In [9]:
# Try out tokenization / detokenization
print(f"Loaded Tokenizer with vocab size of {sp_tokenizer.vocab_size()}")

str = "Hello world!"
tokenized = sp_tokenizer.tokenize(str)
print(f"Tokenized \"{str}\": {tokenized}")

detokenized = sp_tokenizer.detokenize(tokenized)
print(f"Detokenized: {detokenized}")

Loaded Tokenizer with vocab size of 32768
Tokenized "Hello world!": [3869  190   46    2]
Detokenized: b'Hello world!'


In [25]:
import decode
import os
import jax

os.environ["TFDS_DATA_DIR"] = pyconfig.config.dataset_path + "/"
decode.decode_loop(config) 
jax.distributed.shutdown()

Creating checkpoint manager...
Checkpoint manager created!
Devices: [TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0), TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0), TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0), TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)] (num_devices: 4)
Decided on mesh: [[[TpuDevice(id=0, process_index=0, coords=(0,0,0), core_on_chip=0)
   TpuDevice(id=2, process_index=0, coords=(0,1,0), core_on_chip=0)
   TpuDevice(id=1, process_index=0, coords=(1,0,0), core_on_chip=0)
   TpuDevice(id=3, process_index=0, coords=(1,1,0), core_on_chip=0)]]]
Model path: /home/patflick/dev/git/maxtext/assets/tokenizer
restoring state from gs://test-example-123/maxtest-lg/2b-c4/v4-32/maxtext_cnndm/checkpoints latest step 16000


In [61]:
# 
import jax
jax.distributed.shutdown()